## Preprocessing + GPT MODELING 


In [ ]:
import pandas as pd 
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import DutchStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import tiktoken
import json
from openai import OpenAI
import time
import psutil
import tracemalloc
from bertopic import BERTopic
import nbformat
import plotly.express as px
import plotly.graph_objects as go
import umap as UMAP
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')



In [ ]:
client = OpenAI(api_key='xxxxxxxxxxxxxxx')


In [ ]:
##Load the data 
r1 = pd.read_csv('XXXXX', sep=';')
r2 = pd.read_csv('XXXXX', sep=';')
r3 = pd.read_csv("XXXXX", sep=';')
r4 = pd.read_csv('XXXXXX', sep=';')
r5 = pd.read_csv("XXXXXX.csv", sep=';')

#Merge positive and negative together into review column
r3['Review'] = r3['Negatief'].fillna(r3['Positief'])
r4['Review'] = r4['Negatief'].fillna(r4['Positief'])
r5['Review'] = r5['Negatief'].fillna(r5['Positief'])

##make dataframes with only review columns 
r1 = r1[['Review']]
r2 = r2[['Review']]
r3 = r3[['Review']]
r4 = r4[['Review']]
r5 = r5[['Review']]

In [ ]:
####sample and clean reviews
def cleanSample_reviews(df, review_col, sample_size=20000):
    
    # Remove NA values
    df_cleaned = df.dropna(subset=[review_col])
    
    # Randomly sample reviews
    df_sampled = df_cleaned.sample(n=sample_size, random_state=5, replace=True) if len(df_cleaned) > sample_size else df_cleaned
    
    return df_sampled
#####
r1 = cleanSample_reviews(r1, 'Review')
r2 = cleanSample_reviews(r2, 'Review')
r3 = cleanSample_reviews(r3, 'Review')
r4 = cleanSample_reviews(r4, 'Review')
r5 = cleanSample_reviews(r5, 'Review')

In [ ]:

#####PREPROCESSING FUNCTION 

def preprocess_text(text, use_stemming=True):
    # 1. Tokenize the text
    tokens = word_tokenize(text, language='dutch')
    
    # 2. Remove stopwords
    stop_words = set(stopwords.words('dutch'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # 3. Remove numbers and URLs
    tokens = [re.sub(r'\b\d+\b', '', word) for word in tokens]  # Remove numbers
    tokens = [re.sub(r'http\S+|www\S+|https\S+', '', word) for word in tokens]  # Remove URLs
    tokens = [word for word in tokens if word]  # Remove empty strings after removal
    
    # 4. Normalize the text to lowercase
    tokens = [word.lower() for word in tokens]
    
    # 5. Perform stemming or lemmatization
    if use_stemming:
        stemmer = SnowballStemmer('dutch')
        tokens = [stemmer.stem(word) for word in tokens]
    else:
        lemmatizer = WordNetLemmatizer()  # WordNetLemmatizer doesn't support Dutch, so custom implementation needed
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a single string
    processed_text = ' '.join(tokens)
    
    return processed_text



In [ ]:
#Apply preprocessing on the reviews 
r1['P_review'] = r1['Review'].apply(preprocess_text)
r2['P_review'] = r2['Review'].apply(preprocess_text)
r3['P_review'] = r3['Review'].apply(preprocess_text)
r4['P_review'] = r4['Review'].apply(preprocess_text)
r5['P_review'] = r5['Review'].apply(preprocess_text)

In [ ]:
r5.to_csv('R5FINALPROCESS.csv', index=False)
r4.to_csv('R4FINALPROCESS.csv', index=False)
r3.to_csv('R3FINALPROCESS.csv', index=False)
r2.to_csv('R2FINALPROCESS.csv', index=False)
r1.to_csv('R1FINALPROCESS.csv', index=False)

### Word Count per review distribution 

In [ ]:
r1 = pd.read_csv('R1FINALPROCESS.csv')
r2 = pd.read_csv('R2FINALPROCESS.csv')
r3 = pd.read_csv('R3FINALPROCESS.csv')
r4 = pd.read_csv('R4FINALPROCESS.csv')
r5 = pd.read_csv('R5FINALPROCESS.csv')

In [ ]:
all_reviews = pd.concat([r1, r2, r3, r4, r5])
# Apply preprocessing on the reviews
#all_reviews['P_review'] = all_reviews['Review'].apply(preprocess_text)
all_reviews['Review'] = all_reviews['Review'].fillna('').astype(str)
all_reviews['P_review'] = all_reviews['P_review'].fillna('').astype(str)
# Calculate the original and processed word counts
all_reviews['original_word_count'] = all_reviews['Review'].apply(lambda x: len(x.split()))
all_reviews['processed_word_count'] = all_reviews['P_review'].apply(lambda x: len(x.split()))
max_word_count = max(all_reviews['original_word_count'].max(), all_reviews['processed_word_count'].max())




# Plotting the distribution
plt.figure(figsize=(12, 6))

# Overlay histograms with edge color for better definition
plt.hist(all_reviews['original_word_count'], bins=range(0, 80), color='blue', alpha=0.5, edgecolor='black', label='Original', linewidth=1.2)
plt.hist(all_reviews['processed_word_count'], bins=range(0, 80), color='green', alpha=0.5, edgecolor='black', label='Processed', linewidth=1.2)

# Adding labels, title, and grid
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.title('Distribution of Word Counts (Original vs Processed)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Show plot
plt.show()


In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming r1, r2, r3, r4, and r5 are your DataFrames
all_reviews = pd.concat([r1, r2, r3, r4, r5])

# Apply preprocessing on the reviews
# all_reviews['P_review'] = all_reviews['Review'].apply(preprocess_text)
all_reviews['Review'] = all_reviews['Review'].fillna('').astype(str)
all_reviews['P_review'] = all_reviews['P_review'].fillna('').astype(str)

# Calculate the original and processed word counts
all_reviews['original_word_count'] = all_reviews['Review'].apply(lambda x: len(x.split()))
all_reviews['processed_word_count'] = all_reviews['P_review'].apply(lambda x: len(x.split()))

# Combine all text in the 'Review' column into a single string
original_text = ' '.join(all_reviews['Review'])

# Combine all text in the 'P_review' column into a single string
processed_text = ' '.join(all_reviews['P_review'])

# Generate word cloud for original text
wordcloud_original = WordCloud(width=800, height=400, background_color='white').generate(original_text)

# Generate word cloud for processed text
wordcloud_processed = WordCloud(width=800, height=400, background_color='white').generate(processed_text)

# Plotting the word cloud for original text
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_original, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud - Original Text')
plt.show()

# Plotting the word cloud for processed text
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_processed, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud - Processed Text')
plt.show()



### Word cloud for all reviews; before and after 

#### Topic Modelling using GPT API

In [ ]:

# Function to split text into chunks based on the model's token limit
def split_into_chunks(text, max_tokens=16000):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")
    words = text.split()
    chunks = []
    current_chunk = []
    current_tokens = 0

    for word in words:
        word_tokens = len(encoding.encode(word))
        if current_tokens + word_tokens <= max_tokens:
            current_chunk.append(word)
            current_tokens += word_tokens
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_tokens = word_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Topic modeling function using GPT-3.5-turbo
def topic_modeling(example):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
      {
        "role": "system",
        "content": "You are an expert in topic modeling customer review data. I am going to provide you with a dataset of customer reviews. Please analyze this data and return a detailed mapping of the most recurring topics. For each topic, list the most associated keywords that represent it. Ensure that the topics and keywords are clear, distinct, and capture the essence of the reviews."  
      },
      {
        "role": "user",
        "content": f"{example}"
      }
    ],
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  try:
    return response.choices[0].message.content
  except (KeyError, IndexError) as e:
    return "Error: Unable to process the response."





In [ ]:


# Helper function to process each review set
def process_reviews(reviews, index):
    text_data = reviews['P_review'].tolist()
    markdown_block = ', '.join(text_data)
    chunks = split_into_chunks(markdown_block, max_tokens=16000)

    # Get the current process
    process = psutil.Process()

    # Measure initial CPU times and memory usage
    cpu_times_before = process.cpu_times()
    memory_usage_before = process.memory_info().rss

    # Start measuring peak memory usage
    tracemalloc.start()

    # Measure start time
    start_time = time.perf_counter()

    # Apply topic modeling to each chunk
    results = [topic_modeling(chunk) for chunk in chunks]

    # Measure end time
    end_time = time.perf_counter()

    # Measure final memory usage
    memory_usage_after = process.memory_info().rss

    # Stop measuring peak memory usage and get the result
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Measure final CPU times
    cpu_times_after = process.cpu_times()

    # Calculate execution time
    execution_time = end_time - start_time

    # Calculate CPU usage
    user_time = cpu_times_after.user - cpu_times_before.user
    system_time = cpu_times_after.system - cpu_times_before.system
    cpu_usage = ((user_time + system_time) / execution_time) * 100 / psutil.cpu_count()

    # Calculate memory usage difference
    memory_usage_difference = (memory_usage_after - memory_usage_before) / (1024 ** 2)  # Convert bytes to MB

    # Calculate peak memory usage
    memory_usage_peak = peak / (1024 ** 2)  # Convert bytes to MB

    # Print the results
    print(f"Execution time for r{index}: {execution_time} seconds")
    print(f"CPU usage during execution for r{index}: {cpu_usage}%")
    print(f"Peak memory usage during execution for r{index}: {memory_usage_peak} MB")
    print(f"Memory usage difference during execution for r{index}: {memory_usage_difference} MB")

     ###Create DataFrame and save to HTML
    df = pd.DataFrame(results, columns=['topics'])
    df.to_html(f'retailer{index}.html')



In [ ]:
# Process reviews for r1
process_reviews(r1, 1)


In [ ]:
# Process reviews for r2
process_reviews(r2, 2)


In [ ]:


# Process reviews for r3
process_reviews(r3, 3)



In [ ]:

# Process reviews for r4
process_reviews(r4, 4)



In [ ]:
# Process reviews for r5
process_reviews(r5, 5)